In [7]:
keyword = 'data science'
date = ('2018-01-01', '2018-03-31')
page_scroll = 20
username = ''
password = ''

In [8]:
def twitter_scrape(keyword, date, page_scroll, username, password):
    """
    (keyword = int, date = tuple, pageroll = int, username = str, password = str) -> pd.DataFrame
    
    keyword: the word you want to search
    date: date[0] is the starting date  date[1] is the end date
    page_scroll: the number of pages you want to scroll (the function must see the whole content before scraping)
    username: your twitter username
    password; your twitter password
    """
    
    SCROLL_PAUSE_TIME = 2
    
    from selenium import webdriver
    from bs4 import BeautifulSoup
    import time
    import pandas as pd
    
    # Open Browser
    driver = webdriver.Chrome(executable_path = '/usr/bin/chromedriver')
    headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36"}
    
    # Log in Twitter Account
    url = 'https://twitter.com/login'
    driver.get(url)
    username_box = driver.find_element_by_css_selector('#page-container > div > div.signin-wrapper > form > fieldset > div:nth-child(2) > input')
    password_box = driver.find_element_by_css_selector('#page-container > div > div.signin-wrapper > form > fieldset > div:nth-child(3) > input')
    username_box.send_keys(username)
    password_box.send_keys(password)
    driver.find_element_by_css_selector('#page-container > div > div.signin-wrapper > form > div.clearfix > button').click()
    
    # Search Keyword
    search_entry = keyword + ' since:' + date[0] + ' until:' + date[1]
    search_box = driver.find_element_by_css_selector('#search-query')
    search_box.send_keys(search_entry)
    driver.find_element_by_css_selector('#global-nav-search > span > button').click()
    
    # Scroll Pages
    for page in range(page_scroll):
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
    # Parse Content and Fetch Data
    data = pd.DataFrame({'content': [], 'usr': [], 'account_id': [], 'date': [], 'comment': [], 'retweet': [], 'favorite': [], 'link': []})
    souped_content = BeautifulSoup(driver.page_source)
    driver.quit()
    
    streams = souped_content.find_all('div', {'class': 'content'})
    for stream in streams:
        header = stream.find('div', {'class': 'stream-item-header'})
        if header is None:
            continue
        usr = header.find('span', {'class': "FullNameGroup"}).text.strip()
        account_id = header.find('a', {'class': "account-group js-account-group js-action-profile js-user-profile-link js-nav"}).attrs['data-user-id']
        date = int(header.find('span', {'class': "_timestamp js-short-timestamp "}).attrs['data-time-ms'])
        link = header.find('a', {'class': "tweet-timestamp js-permalink js-nav js-tooltip"}).attrs['href']
        content = stream.find('div', {'class': 'js-tweet-text-container'}).text.strip()
        footer = stream.find('div', {'class': 'stream-item-footer'})
        try: 
            comment = footer.find_all('span', {'class': 'ProfileTweet-actionCount'})[0].attrs['data-tweet-stat-count']
        except:
            comment = 0
        try:
            retweet = footer.find_all('span', {'class': 'ProfileTweet-actionCount'})[1].attrs['data-tweet-stat-count']
        except:
            retweet = 0
        try:
            favorite = footer.find_all('span', {'class': 'ProfileTweet-actionCount'})[2].attrs['data-tweet-stat-count']
        except:
            favorite = 0
        data = data.append({'content': content, 'usr': usr, 'date': date, 'favorite': favorite, 'retweet': retweet, 'comment': comment, 'link': link, 'account_id': account_id}, ignore_index=True)
    return data